# Verfahrensergebnisse pro Vorinstanz

strafrechtliche (6B) und strafprozessuale (1B) Bundesgerichtsurteile

von 1.1.2011 bis 30.5.2021

*Die Auswertung basiert auf einer Grundlage, die nicht frei von Fehlern ist!*


In [ ]:
import pandas as pd

jahre = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
kammern = ['1B', '6B']

def urteilsdatenframe_erstellen(kammern, jahre):
    df_list = []
    for i in range(len(kammern)):
        for y in range(len(jahre)):
            df = pd.read_csv('/kaggle/input/bger6b2018/'+ kammern[i] + '_' + str(jahre[y]) + '.csv')
            df = df.drop(columns=['Unnamed: 0'])
            df_list.append(df)
    big_list = pd.concat(df_list)
    return big_list

alle = urteilsdatenframe_erstellen(kammern, jahre)

# Urteil mit falschem Datum korrigieren
alle.loc[alle.Verfahrensnummer == '6B_844/2018', 'Urteilsdatum'] = '13.09.2019'

alle['Urteilsdatum'] = pd.to_datetime(alle['Urteilsdatum'], errors='coerce', format='%d.%m.%Y')

max_dt = alle["Urteilsdatum"].max().strftime('%d.%m.%Y')
max_dt_nr = alle.loc[alle["Urteilsdatum"] == max_dt, 'Verfahrensnummer']

print(f'Auswertung von insgesamt '+ str(alle.shape[0]) + ' online veröffentlichten Urteilen. Das jüngste Urteil datiert vom ' + str(max_dt) + ' (' + str(max_dt_nr.values) + ').')
 

# Datenbereinigung

Korrektur von **Vorinstanzen**

In [ ]:
vorinstanz_error = alle[alle['Vorinstanz'].str.contains("#")]
print(f'Bei ' + str(vorinstanz_error.shape[0]) + ' Urteilen konnte der Auswertungsalgorithmus die Vorinstanz nicht eruieren und gab eine Fehlermeldung zurück. Diese Lücken werden in der Folge mit einem verifizierten Korrekturdatensatz korrigert.')

In [ ]:
def vorinstanzkorrektur(pfad=f'/kaggle/input/bger6b2018/val_sets/straf/vorinstanzkorrektur.csv'):
    try:
        korr_vor = pd.read_csv(pfad)
        print(f'Von {pfad} wurden {korr_vor.shape[0]} Vorinstanzkorrekturen eingelesen.')
        korrekturcount = 0
        for i in range(korr_vor.shape[0]):
            urteilsnummer = korr_vor.iloc[i, 1]
            if urteilsnummer in alle['Verfahrensnummer'].unique():
                korrekturcount += 1
                vorinstanz = korr_vor.iloc[i, 2]
                alle.loc[alle.Verfahrensnummer == urteilsnummer, 'Vorinstanz'] = vorinstanz
            else:
                pass
        print(f'Im Zieldatenframe wurden bei {korrekturcount} Urteilen die Vorinstanz korrigiert')
    except FileNotFoundError:
        print(f'Pfad {pfad} existiert nicht, daher erfolgte keine Vorinstanzkorrektur')
        pass

vorinstanzkorrektur()

In [ ]:
vorinstanz_error = alle[alle['Vorinstanz'].str.contains("#")]
if vorinstanz_error.shape[0] > 0:
    print(f'Es verbleiben ' + str(vorinstanz_error.shape[0]) + ' Urteile mit einem Vorinstanz-Auswertungsfehler. Die sind neu hinzugekommen und wurden noch nicht händisch korrigiert.')
else:
    print('Es verbleiben keine Urteile mehr im Datensatz, in welchem keine Vorinstanz festgelegt ist.')

Im folgenden werden Urteile, bei denen der Auswertungsalgorithmus nicht ein fehlerhaftes, aber merkwürdiges Vorinstanzresultat ergab, händisch korrigert. Eine Liste mit verbliebenen fehlerhaft erfassten oder seltenen Vorinstanzen, die bei der folgenden Auswertung nicht berücksichtigt wurden, finden sich ganz am Ende.

In [ ]:
# Regex-Ersetzungen
alle = alle.replace(r'(?<=Anklagekammer des Kantons St. Gallen).*', '', regex=True)
alle = alle.replace(r'Anklagekammer.*Gallen', 'Anklagekammer des Kantons St. Gallen', regex=True)
alle = alle.replace(r'(?<=Anklagekammer des Kantons Thurgau).*', '', regex=True)
alle = alle.replace(r'(?<=Anklagekammer des Kantons Waadt).*', '', regex=True)
alle = alle.replace(r'(?<=Appellationsgericht des Kantons Basel-Stadt).*', '', regex=True)
alle = alle.replace(r'(?<=Appellationsgerichtpr).*', 'aesidentIn des Kantons Basel-Stadt', regex=True)
#alle = alle.replace(r'Cour de cassation pénale', 'Tribunal cantonal du canton de Vaud', regex=True)
#alle = alle.replace(r'Tribunal cantonal', "Tribunal d'accusation du Tribunal cantonal du canton de Vaud", regex=True)

Korrektur von **Ergebnissen**

In [ ]:
ergebnis_error = alle[alle['Verfahrensergebnis'].str.contains('#')]
fehlerzahl = ergebnis_error.shape[0]
print(f'Bei {fehlerzahl} Urteilen konnte der Auswertungsalgorithmus das Ergebnis nicht feststellen und gab eine Fehlermeldung zurück. Die betreffenden Urteile werden in der Folge (teilweise) mit einem verifizierten Korrekturdatensatz korrigiert.')

In [ ]:
def ergebniskorrektur(pfad='/kaggle/input/bger6b2018/val_sets/straf/ergebniskorrektur.csv'):
    try:
        korr_erg = pd.read_csv(pfad)
        print(f'Von {pfad} wurden {korr_erg.shape[0]} Ergebniskorrekturen eingelesen.')
        korrekturcount = 0
        for i in range(korr_erg.shape[0]):
            urteilsnummer = korr_erg.iloc[i, 1]
            if urteilsnummer in alle['Verfahrensnummer'].unique():
                korrekturcount += 1
                ergebnis = korr_erg.iloc[i, 2]
                alle.loc[alle.Verfahrensnummer == urteilsnummer, 'Verfahrensergebnis'] = ergebnis
            else:
                pass
        print(f'Im Zieldatenframe wurden bei {korrekturcount} Urteilen das Verfahrensergebnis korrigiert')
    except FileNotFoundError:
        print(f'Pfad {pfad} existiert nicht, daher erfolgte keine Ergebniskorrektur')
        pass
    
ergebniskorrektur()

In [ ]:
ergebnis_error = alle[alle['Verfahrensergebnis'].str.contains('#')]
fehlerzahl = ergebnis_error.shape[0]
if fehlerzahl > 0:
    print(f'Es verbleiben {fehlerzahl} Urteile mit einem bei welchen der Ergebnis-Auswertungsalgorithmus einen Fehler zurückgegeben hat.')
else: 
    print('Es verbleiben keine Urteile mehr im Datensatz, in welchem kein Verfahrensergebnis festgelegt ist.')

**Es folgt eine Fehlerquoten-Evaluation**

Der Auswertungsalgorithmus interpretiert die Ergebnisse teilweise falsch (z.B. ein gutgeheissenes UR-Gesuch wird fälschlicherweise als Gutheissung der Beschwerde interpretiert). Um die Fehlerquote hochzurechnen werden in der Folge menschlich verifizierte Samples mit dem durch den Algorithmus errechneten Verfahrensergebnissen abgeglichen und die Übereinstimmungsquote eruiert, womit die Fehlerquote des gesamten Datensatzes abgeschätzt werden kann.

In [ ]:
rechtsbereich = 'straf'

def fehlerquoten_evaluation(val_set_folderpath=f'/kaggle/input/bger6b2018/val_sets/{rechtsbereich}', print_details=False):
    #print('fehlerquoten_evaluation(): ')
    count_dict = dict()
    fehlerquote_dict = dict()
    for val_set in ['de_val_set', 'fr_val_set', 'it_val_set']:
        filepath = f'{val_set_folderpath}/{val_set}.csv'
        try:
            val_df = pd.read_csv(filepath)
            #print(f' Von {filepath} insgesamt {str(val_df.shape[0])} evaluierte Verfahrensergebnisse eingelesen.')
            val_df = pd.merge(alle, val_df, on='Verfahrensnummer', how='inner')
            if val_set == 'de_val_set':
                sprache = 'deutschen'
            elif val_set == 'fr_val_set':
                sprache = 'französischen'
            else:
                sprache = 'italienischen'
            print(f'In diesem Datenset können insgesamt {val_df.shape[0]} Ergebnisse von {sprache} Urteilen kontrolliert werden.')
            count_dict[val_set] = val_df.shape[0]
            matches = 0
            dismatches = 0
            for row in range(val_df.shape[0]):
                if val_df.iloc[row]['Verfahrensergebnis'] == val_df.iloc[row]['Ergebnis']:
                    matches += 1
                else:
                    dismatches += 1
                    if print_details:
                        print(f' -> Beim Urteil {val_df.iloc[row]["Verfahrensnummer"]} hat der Algorithmus '
                              f'fälschlicherweise auf "{val_df.iloc[row]["Verfahrensergebnis"]}" erkannt, das '
                              f'korrekte Ergebnis wäre: "{val_df.iloc[row]["Ergebnis"]}".')

            fehlerquote = dismatches/(matches+dismatches)*100
            print(f' Die Fehlerquote im {sprache} Evaluationsset beträgt {str(fehlerquote)}%')
            fehlerquote_dict[val_set] = fehlerquote/100
        except FileNotFoundError:
            print(f' {filepath} existiert nicht')
    count_list = list(count_dict.values())
    fehlerquoten_list = list(fehlerquote_dict.values())
    count_fehlerquote_tuple = list(zip(count_list, fehlerquoten_list))
    gesamtcount = sum(count_list)
    print(f'Insgesamt wurde das Ergebnis des Auswertungslogarithmus bei {gesamtcount} verifizierten Urteilen überprüft')
    gewichtete_fehlerquoten = [tup[0]*tup[1]/gesamtcount for tup in count_fehlerquote_tuple]
    gesamtfehlerquote = sum(gewichtete_fehlerquoten)
    print(f' Dabei wurde eine Fehlerquote von {round(gesamtfehlerquote*100, 2)}% festgestellt.')
     
fehlerquoten_evaluation(print_details=False)

# Gutheissungsquoten im interkantonalen Vergleich (alle strafrechtlichen Bundesgerichtsurteile ab 2011)

In [ ]:
counted = alle.groupby(["Vorinstanz", "Verfahrensergebnis"]).count()
counted = counted.reset_index()

# ZMG-Vorinstanzen ausfiltern, "|" bedeutet oder
counted = counted[counted['Vorinstanz'].str.contains('ZMG|mesures|Bezirksgericht|undefiniert')==False]

pivot = counted.pivot(values="Verfahrensnummer", index="Vorinstanz", columns="Verfahrensergebnis")
pivot["Total"] = pivot.sum(axis=1)
pivot = pivot.fillna(0)
pivot["% gut"] = (pivot["Gutheissung"]+pivot["teilweise Gutheissung"])/pivot["Total"]*100
pivot["% gut-mat"] = (pivot["Gutheissung"]+pivot["teilweise Gutheissung"])/(pivot["Total"]-pivot["Nichteintreten"]-pivot["Gegenstandslosigkeit"])*100
filtered = pivot[pivot["Total"] > 19]
sortiert = filtered.sort_values("% gut-mat", ascending=False)
# Spaltennamen kürzen
sortiert = sortiert.rename(columns={"Abweisung": "abw", "Gegenstandslosigkeit": "gglos", "Gutheissung": "gut", "Nichteintreten": "nee", "teilweise Gutheissung": "tlwgut", "Total": "tot"})
# Spaltennamen neu einordnen
sortiert = sortiert[['tot', 'gut', 'tlwgut','abw', 'nee', 'gglos', '% gut', '% gut-mat']]
sortiert['tot-mat'] = sortiert['tot'] - sortiert['nee'] - sortiert['gglos']
sortiert['tot-gut'] = sortiert['gut'] + sortiert['tlwgut']
sortiert = sortiert[['tot', 'tot-mat', 'tot-gut', 'gut', 'tlwgut', 'abw', 'nee', 'gglos', '% gut', '% gut-mat']]

#funktion um Spalten hervorzuheben:
def highlight_cols(x):
    #copy df to new - original data are not changed
    df = x.copy()
    #select all values to default value - none
    df.loc[:,:] = ''
    #overwrite values lightyellow color
    df[['tot']] = 'font-weight: bold'
    #overwrite values lightyellow color
    df[['gut', 'tlwgut', 'tot-gut']] = 'background-color: lightgreen'
    #overwrite values lightyellow color
    df[['tot-gut']] = 'font-weight: bold; background-color: lightgreen'
    #overwrite values lightred color
    df[['abw', 'nee', 'gglos']] = 'background-color: lightyellow'
    #return color df
    return df      

sortiert = sortiert.fillna(0)
sortiert.style.background_gradient(subset=['% gut', '% gut-mat'], cmap='Blues')\
.apply(highlight_cols, axis=None)

In [ ]:
from matplotlib import pyplot as plt

## Graph erstellen
plt.figure(figsize=[14, 4.8])
ax1 = plt.subplot()
ax1.set_xticks(range(len(sortiert['% gut-mat'])))
ax1.set_xticklabels(sortiert.index.to_list(), rotation='vertical')
plt.bar(range(len(sortiert['% gut-mat'])), sortiert['% gut-mat'].to_list())
plt.title('Gutheissungsquote pro Vorinstanz in Prozent im Verhältnis zu allen materiell beurteilten Fällen (Gutheissungen und Abweisungen)')
plt.ylabel('%')
plt.xlabel('Vorinstanz')

plt.show()

**Liste aller verbliebenen Vorinstanzen-Ergebnisse des Auswertungsalgorithmus (nach Datenbereinigung)**

In [ ]:
err_count = alle.groupby(['Vorinstanz']).Verfahrensnummer.count().reset_index().rename(columns={'Verfahrensnummer': 'tot'}).set_index('Vorinstanz')
pd.set_option('display.max_rows', 1000)
err_count.head(1000).sort_values('tot')